In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from geopy.distance import geodesic
import time

file_name = "graph.txt"

with open(file_name, 'r') as file:
    data = file.readlines()

points = []
for line in data:
    components = line.strip().split(',')
    point = (float(components[0]), float(components[1]))
    points.append(point)

G = nx.Graph()
G.add_nodes_from(range(len(points)))
for i in range(len(points) - 1):
    distance_km = geodesic(points[i], points[i + 1]).kilometers
    G.add_edge(i, i + 1, weight=distance_km)

pos = {i: (points[i][1], points[i][0]) for i in range(len(points))}
edge_labels = nx.get_edge_attributes(G, 'weight')

plt.figure(figsize=(10, 6))
nx.draw(G, pos, with_labels=True, font_weight='bold', node_size=800, node_color='green', font_size=8,
        edge_color='black', alpha=0.7)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='yellow', font_size=8)

start_node = 2
end_node = 9

def heuristic(node):
    h = abs(pos[node][0] - pos[end_node][0]) + abs(pos[node][1] - pos[end_node][1])
    return h

def recursive_best_first_search(graph, current_node, goal, f_limit, heuristic_func):
    if current_node == goal:
        return [current_node], 0

    successors = []
    for neighbor in graph.neighbors(current_node):
        cost = graph[current_node][neighbor]['weight']
        h = heuristic_func(neighbor)
        successors.append((neighbor, cost, h))

    while successors:
        successors.sort(key=lambda x: x[2])  # Sort by heuristic value
        best_node, best_cost, best_h = successors[0]

        if best_h > f_limit:
            return None, best_h

        alternative = successors[1][2] if len(successors) > 1 else float('inf')

        result, new_f_limit = recursive_best_first_search(graph, best_node, goal, min(f_limit, alternative), heuristic_func)

        if result is not None:
            return [current_node] + result, best_cost

        successors[0] = (best_node, best_cost, new_f_limit)

    return None, float('inf')

def rbfs(graph, start, end, heuristic_func):
    start_time = time.time()
    path_nodes, cost = recursive_best_first_search(graph, start, end, float('inf'), heuristic_func)
    end_time = time.time()
    return path_nodes, cost, end_time - start_time

rbfs_path_nodes, rbfs_cost, rbfs_execution_time = rbfs(G, start_node, end_node, heuristic)

total_cost_rbfs = sum(G[rbfs_path_nodes[i]][rbfs_path_nodes[i + 1]]['weight'] for i in range(len(rbfs_path_nodes) - 1))

print(f"Start node: {start_node}")
print(f"End node: {end_node}")
print(f"RBFS Shortest path: {rbfs_path_nodes}")
print(f"RBFS Total cost: {total_cost_rbfs:.2f} km")
print(f"RBFS Total time: {rbfs_execution_time:.6f} seconds")
